In [ ]:
from sympy import *
s, t = symbols('s t')
f = (abs(s - 0.5) + 1)/(s-t+2)
r = integrate(f, (s, 0, 3))
simplify(r)

In [ ]:
import numpy as np
from math import *
import matplotlib.pyplot as plt

In [ ]:
def K(t, s):
    return 1 / (s - t + 2)

def qe(s):
    return abs(s - 0.5) + 1

def f(t):
    return 3.5*(0.285714285714286*t-1.0)*log(2.0-t)-3.5*(0.285714285714286*t-1.0)*log(2.5-t)-1.5*(0.666666666666667*t-1.0)*log(2.5-t)+1.5*(0.666666666666667*t-1.0)*log(5.0-t)+2.0

ls = 3
lt = 1

Ns = [int(pow(2,i)) for i in range(1,11)]

def _A(N):
    hx = ls / N
    A = np.zeros((N+1, N+1))
    for j in range(N+1):
        tj = 0 + j*(lt/N)
        for i in range(1,N):
            si = 0 + i*(ls/N)
            A[j,i] = hx * K(tj, si)
        A[j,0] = 0.5 * hx * K(tj, 0)
        A[j,N] = 0.5 * hx * K(tj, ls)
    return A

def _b(N):
    b = np.zeros((N+1,1))
    for j in range(N+1):
        tj = 0 + j*(lt/N)
        b[j,0] =  f(tj)
    return b

def solve(A, b):
    U, S, V = np.linalg.svd(A, full_matrices=True)
    
    UTb = np.transpose(U) @ b 
    S1UTb = UTb
    for i in range(len(S1UTb)):
        if(S[i] != 0):
            S1UTb[i] = S1UTb[i] / S[i]
        else:
            S1UTb[i] = 0
    return np.transpose(V) @ S1UTb

# 1

In [ ]:
x = np.linspace(0,ls, 1000)
y = [qe(p) for p in x]

bAdd = [(qe(0) * (1000 - i) + i * qe(ls)) / 1000 for i in range(1000) ]
    
plt.plot(x, y, label = "Точное решение")
plt.plot(x, bAdd, label = "Априорная информация")
plt.legend()
plt.xlabel('s')
plt.ylabel('q(s)')
plt.grid(True)
plt.show()

In [ ]:
x = np.linspace(0,lt, 1000)
y = [f(p) for p in x]

plt.plot(x, y, label = "f(t)")
plt.legend()
plt.xlabel('t')
plt.ylabel('f(t)')
plt.grid(True)
plt.show()

In [ ]:
x = np.linspace(0,ls, 1000)
y = [qe(p) for p in x]

for n in Ns:
    A = _A(n)
    b = _b(n)
    qsol = solve(A,b)
    
    plt.plot([0 + i * ls/n for i in range(0, n+1)], qsol, label = "Приближенное решение")
    plt.plot(x, y, label = "Точное решение")
    
    plt.legend()
    plt.title("График приближенного решения при N = " + str(n))
    plt.xlabel('s')
    plt.ylabel('q(s)')
    plt.grid(True)
    plt.show()

In [ ]:
Er = []
Er1 = []
for n in Ns:
    A = _A(n)
    b = _b(n)
    
    qexs = np.zeros((n+1,1))
    for i in range(n+1):
        qexs[i,0] =  qe(0 + i*(ls/n))

    qsol = solve(A,b)
    
    Er.append((np.transpose(A@qexs - b) @ (A@qexs - b))[0,0] / n)
    Er1.append((np.transpose(A@qsol - b) @ (A@qsol - b))[0,0] / n)

plt.plot(Ns, np.log(Er)/np.log(10), label = "MSE точного решения")
plt.plot(Ns, np.log(Er1)/np.log(10), label = "MSE полученного решения")
plt.legend()
plt.xlabel('N')
plt.ylabel('log MSE')
plt.grid(True)
plt.show()

# 2

In [ ]:
S1 = []
S2 = []
for n in Ns:
    A = _A(n)
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    S1.append(s[0])
    S2.append(s[n])
    
plt.plot(Ns, np.log(S1)/np.log(10), label = "Smax")
plt.plot(Ns, np.log(S2)/np.log(10), label = "Smin")
plt.legend()
plt.title("Масксимальное и минимальное синглуярные числа")
plt.xlabel('N')
plt.ylabel('log S')
plt.grid(True)
plt.show()

plt.plot(Ns, [np.log(S1[i]/S2[i])/np.log(10) for i in range(len(S2))], label = "Число обусловленности")
plt.legend()
plt.title("Зависимость числа обсуловленности от размера сетки")
plt.xlabel('N')
plt.grid(True)
plt.show()

In [ ]:
n = 1024
A = _A(n)
u, s, vh = np.linalg.svd(A, full_matrices=True)
plt.plot(list(range(A.shape[0])), np.log(s)/np.log(10), label = "Si", marker = ".")
plt.legend()
plt.title("Сингулярные числа матрица при N = " + str(n))
plt.xlabel('Номер сингулярного числа')
plt.grid(True)
plt.show()

# 3

In [ ]:
n = 20

noiseLevels = [0.0, 0.01,0.03, 0.05, 0.1]
A = _A(n)
b = _b(n)
x = np.linspace(0, ls, n+1)
qexs = np.zeros((n+1,1))
for i in range(n+1):
    qexs[i,0] =  qe(0 + i*(ls/n))

noise = np.transpose(np.random.rand(len(b))*2-1)    

for noiseLevel in noiseLevels:
    norm = np.linalg.norm(b)
    bD = b + (noise * norm * noiseLevel).reshape(b.shape)
    
    qsol = solve(A,bD)

    mse = ((np.transpose((qsol - qexs)) @ (qsol - qexs)) / n)[0,0]
    #print(mse)
    plt.plot(x, qsol, label = "Qsol")
    plt.plot(x, qexs, label = "Qex")
    plt.legend()
    plt.title("Уровень шума = " + str(noiseLevel) + "; MSE = " + str(mse))
    plt.xlabel('N')
    plt.grid(True)
    plt.show()

# 4